# url, date, title 데이터 가져오기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests
import time
import pandas as pd

In [2]:
def naver_crawling(result, naver_url):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)
    driver.get(naver_url)

    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 웹페이지 맨 아래까지 무한 스크롤
    while True:        
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        curr_height = driver.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # url, date, title
    contents = driver.find_elements_by_css_selector('div.total_area')

    for content in contents:
        url = content.find_element_by_css_selector('a.api_txt_lines.total_tit').get_attribute('href')
        date = content.find_element_by_css_selector('span.sub_time.sub_txt').text
        title = content.find_element_by_css_selector('a.api_txt_lines.total_tit').text

        result.append([url, date, title])
        
    driver.quit()
        
    return result

In [3]:
# url, date, title 정보
result = []

# 20170101 ~ 20180731
keyword = "웰니스관광"
naver_url = "https://search.naver.com/search.naver?where=blog&query="+ quote(keyword) + "&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20170101to20180731"
result = naver_crawling(result, naver_url)

# 20180801 ~ 20200131
naver_url = "https://search.naver.com/search.naver?where=blog&query="+ quote(keyword) + "&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20180801to20200131"
result = naver_crawling(result, naver_url)

print(len(result))

<ipython-input-2-e3966ea796fe>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)


2000


In [4]:
naver_df = pd.DataFrame(result)
naver_df.columns = ['url', 'date', 'title']

# hashtag, 글내용 데이터 가져오기

In [9]:
hashtags = []
contents = []
i = 0

options = webdriver.ChromeOptions()
options.add_argument("headless")
    
driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)

for blog_url in naver_df['url']:
    hashtag_list = []
    content_list = []
    
    # 몇번째인지 확인하는 코드, 추후 삭제 예정
    i += 1
    if i % 20 == 0:
        print(str(i) + "번째")
    
    try:
        driver.get(blog_url)
        time.sleep(5)
        
    except:
        hashtags.append(hashtag_list)
        contents.append(content_list)
        continue
    
    # 해시태그 크롤링
    try:
        driver.switch_to.frame("mainFrame")
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

    except:
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
    
    try:
        hashtag_data = soup.select("a.item.pcol2.itemTagfont._setTop")
    
        for hashtag in hashtag_data:
            hashtag_list.append(hashtag.text)
    
    except:
        hashtag_list.append("에러")

    hashtags.append(hashtag_list)
    
    # 내용 크롤링
    try:
        content_data = soup.select("div.se-component.se-text.se-l-default")

        for content in content_data:
            content_list.append(content.text)
        
        if len(content_list) == 0:
            content_data = soup.select("div.post-view p")
            
            for content in content_data:
                content_list.append(content.text)
        
        if len(content_list) == 0:
            content_data = soup.select("div.se_component_wrap.sect_dsc.__se_component_area p")
            
            for content in content_data:
                content_list.append(content.text)
                
        if len(content_list) == 0:
            content_data = soup.select("div.post-view div")

            for content in content_data:
                content_list.append(content.text)
        
    except:
        content_list.append("에러")
        
    contents.append(content_list)
    
    # 추후 삭제 예정
    if i % 500 == 0:
        print("에러방지용 sleep 진행중")
        time.sleep(180)
    
driver.quit()

<ipython-input-9-8316ee234a93>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)


20번째
40번째
60번째
80번째
100번째
120번째
140번째
160번째
180번째
200번째
220번째
240번째
260번째
280번째
300번째
320번째
340번째
360번째
380번째
400번째
420번째
440번째
460번째
480번째
500번째
에러방지용 sleep 진행중
520번째
540번째
560번째
580번째
600번째
620번째
640번째
660번째
680번째
700번째
720번째
740번째
760번째
780번째
800번째
820번째
840번째
860번째
880번째
900번째
920번째
940번째
960번째
980번째
1000번째
에러방지용 sleep 진행중
1020번째
1040번째
1060번째
1080번째
1100번째
1120번째
1140번째
1160번째
1180번째
1200번째
1220번째
1240번째
1260번째
1280번째
1300번째
1320번째
1340번째
1360번째
1380번째
1400번째
1420번째
1440번째
1460번째
1480번째
1500번째
에러방지용 sleep 진행중
1520번째
1540번째
1560번째
1580번째
1600번째
1620번째
1640번째
1660번째
1680번째
1700번째
1720번째
1740번째
1760번째
1780번째
1800번째
1820번째
1840번째
1860번째
1880번째
1900번째
1920번째
1940번째
1960번째
1980번째
2000번째
에러방지용 sleep 진행중


In [10]:
naver_df['hashtags'] = hashtags
naver_df['contents'] = contents

# 엑셀 파일로 저장하기

In [11]:
naver_df.to_excel("코로나전_" + keyword + '.xlsx', index = False)

In [12]:
naver_df

,url,date,title,hashtags,contents
0,https://blog.naver.com/sendex?Redirect=Log&log...,2017.09.15.,웰니스 관광 어떤 것이 있나 확인해볼까,"[#웰니스, #웰니스관광, #웰니스관광산업, #관광산업, #관광활성화, #웰니스관광...",[안녕하세요~SENDEX 2017 사무국입니다 :)역대 최장 황금연휴가 될 이번 추...
1,https://blog.naver.com/we-korea?Redirect=Log&l...,2017.08.29.,웰니스관광으로 여행과 건강을,"[#웰니스관광, #정책기자단, #정책브리핑]","['여보, 우리 숲 보면서 스파하러 가요~', 웰니스 = 웰빙 + 건강신체적, 정신..."
2,https://blog.naver.com/mcstkorea?Redirect=Log&...,2017.07.26.,[영상] #건강#힐링#여행_웰니스관광! ‘동의보감촌’,"[#건강, #힐링, #한방, #동의보감촌, #웰니스관광]","[, 문화체육관광부는 최근 웰니스 관광이 각광받는 추세를 반영해 관광분야를 한방, ..."
3,https://blog.naver.com/geochanggun?Redirect=Lo...,2018.05.29.,[거창군] 제1기 웰니스관광코디네이터 전문가 양성과정 교육생...,"[#항노화, #웰니스관광코디네이터, #항노화산업, #거창군, #웰빙, #관광코디네이...","[웰니스관광 및 항노화산업에 관심을 갖고, 웰니스관광코디네이터로서 자긍심과 사명감을..."
4,https://blog.naver.com/sinnanie?Redirect=Log&l...,2018.01.12.,[배경지식 넓히기] 명품 '웰니스 관광',"[#시사원정대, #동아이지에듀, #어린이시사, #웰니스관광, #한국관광공사, #진안...","[시사원정대 1월호 시사 똑똑북맞춤사설 배경지식 넓히기명품 '웰니스 관광'!, 요즘..."
...,...,...,...,...,...
1995,https://blog.naver.com/terry-k?Redirect=Log&lo...,2019.09.01.,[강화도땅] 인천시토지 강화도땅 송해면 양오리 호수가 보이는...,"[#강화도땅, #서울강화간고속화도로, #인천시토지, #민통선관광코스, #강화평화전망...","[\n\n\n""요즘 도심이랑 가까우면서도 전원의 생활을 즐기고자 하는 사람들이 많이..."
1996,https://blog.naver.com/touryklee?Redirect=Log&...,2019.04.22.,[이용근교수의 행복여행] 7년만의 화려한 외출 : 제주...,[],"[\n\n\n2010년 스페인 산티아고 걷기여행을 가족여행의 종지부로 찍고, 일상을..."
1997,https://blog.naver.com/mchm2013?Redirect=Log&l...,2019.07.08.,"전남문화관광재단, 글로벌 헬스케어 확대 온힘",[],"[ 전남문화관광재단이 다국적 결혼이민자를 비롯한 귀화자, 유학생을 대상으로 의료..."
1998,https://blog.naver.com/neomgl?Redirect=Log&log...,2019.05.09.,방한 외래관광객… 더 자주오고 오래 머물고 지방방문 늘었다,[#관광],[\n\n\n승인 2019.05.08 13:252018 외래관광객 실태조사 결과 발...
